# Collection of Twitter data for the <font color='red'>#Day-zero</font> sentiment ananlysis
Data is scraped from twitter relating to the waer  crisis issue in Cape Town

- drought sort of began in 2015 in the Western cape
- Cape town Details:
    - -33.9249° S, 18.4241° E
- Countries we do business in:


### Loading Packages

In [3]:
import numpy as np
import pandas as pd
import sqlite3
import json
from tweepy.streaming import StreamListener
# import jsonpickle

### Authotrization

In [120]:
# import access keys
from credentials import *

# API setup
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tpy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    # Return API with authentication:
    api = tpy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)  # todo: include rate limit parameters
    return api

def insert_tweet(tweet, db, table):
    """ inserts tweet object in the table "table" in the database "db".
    
    keyword arguments:
    tweet -- tweet object
    db    -- the database 
    table -- table in database "db"
    """
    conn = sqlite3.connect(db)
    c = conn.cursor()
    print('in insert method')
    with conn:
        c.execute("""INSERT INTO """ + table + """ VALUES (:text, :date, :tweet_language, :retweet_count, 
                                                    :favorite_count, :tweet_id, :follower_count, :friends_count, 
                                                    :location, :user_id, :user_language, :geo_enabled, 
                                                    :verified, :user_location)""",
                  {'text': tweet['text'], 'date': tweet['created_at'], 'tweet_language': tweet['lang'],
                  'retweet_count': tweet['retweet_count'], 'favorite_count': tweet['favorite_count'],
                  'tweet_id': tweet['id_str'], 'follower_count': tweet['user']['follower_count'],
                  'friends_count': tweet['user']['friends_count'], 'location': tweet['user']['location'],
                  'user_id': tweet['user']['id_str'], 'user_language': tweet['user']['lang'],
                  'geo_enabled': tweet['user']['geo_enabled'], 'verified': tweet['user']['verified'],
                  'user_location': tweet['user']['user_location']})

def get_tweet():
    c.execute("SELECT * FROM tweets")
    return c.fetchall()

In [71]:
class TweetStreamListener(StreamListener):

    """ A listener handles tweets that are received from the stream.
    This is a basic listener that just prints received tweets to stdout.
    """
    def on_data(self, data):
        try:
            # process and insert tweet in database
            
            # convert to python dictionary
            tweet = json.loads(data)
            
            # filter retweets and insert into table in database
            if not tweet.retweeted and 'RT @' not in tweet.text:
                
                print("inserting tweet to data base")
                insert_tweet(tweet, 'twitter_data.db', 'tweet')
                print("success inserting tweet")

        except Exception as e:
            print(e)
            print('in exception')
            pass

        return True

In [65]:
conn = sqlite3.connect('twitter_data.db')  # change db name
c = conn.cursor()
# c.execute("DROP TABLE tweets")
c.execute("""
        CREATE TABLE IF NOT EXISTS tweets (
        text TEXT,
        date TEXT,
        tweet_language TEXT,
        retweet_count INTEGER,
        favorite_count INTEGER,
        tweet_id TEXT,
        follower_count INTEGER,
        friends_count INTEGER
        location TEXT,
        user_id TEXT,
        user_lang TEXT,
        geo_enabled NUMERIC,
        verified NUMERIC,
        user_location TEXT
        )""")
print("table created")

table created


In [72]:
api = twitter_setup()
print('api set up done')
l = TweetStreamListener()
stream = tweepy.Stream(api.auth, l)
stream.filter(track=['day zero water', 'water crisis cape', 'rain cape day zero', 'cape town drought'])

api set up done
'dict' object has no attribute 'retweeted'
in exception


KeyboardInterrupt: 

In [126]:
tweet_data['user']['followers_count']

256

In [111]:
conn.close()

In [40]:
[tweet.id, tweet.created_at, tweet.text, tweet.user.followers_count, tweet.user.verified, tweet.possibly_sensitive, 
 tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.entities['hashtags'], tweet.sc ]

[956781542491283456,
 datetime.datetime(2018, 1, 26, 6, 51, 10),
 'RT @RickyPo: Sub-Saharan nations lead the way in open access https://t.co/c7XgMmIZsp',
 684,
 False,
 False,
 None,
 None,
 1,
 0,
 []]

In [59]:
# api.geo_search
api.rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1517220407}}

After perusing a few tweets as well as news articles, the following keywords have been suggested

In [177]:
search_query = ('"day zero" water OR water crisis cape OR "day zero" crisis OR cape water OR cape drought')
max_tweets= 100000
tweets_per_query = 100

In [178]:
#with open('../data/tweets.json', mode='w') as f:  # do a check if file already exists as 'x' creates file each time
date = []
text = []
retweet_count = []
follower_count = []
location = []
tags = []
verified = []

tweets = tpy.Cursor(api.search, q=search_query).items(max_tweets)

for tweet in tweets:
    if hasatt
    text.append(tweet.text)
    date.append(tweet.created_at)
    retweet_count.append(tweet.retweet_count)
    follower_count.append(tweet.user.followers_count)
    location.append(tweet.user.location)
    tags.append(tweet.entities['hashtags'])
    verified.append(tweet.user.verified)
    
tweet_data = pd.DataFrame({'text': text,
                          'date': date,
                          'retweet_count': retweet_count,
                          'follower_count': follower_count,
                          'location': location,
                          'tags': tags,
                          'verified': verified})

,date,follower_count,location,retweet_count,tags,text,verified
0,2018-01-26 09:47:59,247,UK,0,"[{'text': 'water', 'indices': [62, 68]}]","Another commodity, alongside food, that we tak...",False
1,2018-01-26 09:47:59,737,Washington (the state),1,"[{'text': 'CapeTown', 'indices': [21, 30]}]",RT @TheDailyClimate: #CapeTown authorities are...,False
2,2018-01-26 09:45:10,15189,"Bozeman, Mont.",1,"[{'text': 'CapeTown', 'indices': [0, 9]}]",#CapeTown authorities are scrambling to persua...,False
3,2018-01-26 02:14:37,650,"Cape Town, South Africa",147,[],"RT @AssaadRazzouk: Day Zero: Within 4 Months, ...",False
4,2018-01-26 00:00:00,419,Japan,134,[],RT @Reuters: WATCH: Cape Town's authorities ar...,False


In [ ]:
# todo kparse location column nicely

def parse_location(string):
    
    """
    takes in a string and returns the best guess for a country name
    """

In [72]:
# DB stuff
conn = sqlite3.connect('twitter.db')
c = conn.cursor()

class Tweet():
    
    def __init__(self, text, user, followers, date, location):
        self.text = text
        self.user = user
        self.followers = followers
        self.date = date
        self.location = location

        
# Inserting that data into the DB
    def insertTweet(self):

        c.execute("INSERT INTO tweets (tweetText, user, followers, date, location) VALUES (?, ?, ?, ?, ?)",
            (self.text, self.user, self.followers, self.date, self.location))
        conn.commit()

In [75]:
conn.close()

In [124]:
conn = sqlite3.connect('test.db')
c = conn.cursor()
table = 'tweets'
for row in c.execute('SELECT text FROM ' + table + ' LIMIT 3'):
        print(row)

("b'Conspiracy https://t.co/rLZUteTM4S'",)
("b'Meaning farms of cape town are more important than human beings. This shocked\\xf0\\x9f\\x98\\xa8\\xf0\\x9f\\x98\\xa8 https://t.co/cpzzfFiRzE'",)
("b'#DayZero \\xe2\\x80\\x93 the day Cape Town is expected to turn off fresh water supply to suburbs \\xe2\\x80\\x93 is pegged at April 12 https://t.co/8LkSkvAQh5'",)
